# Use Local Earnings Call Transcripts

This notebook shows how to read a local Parquet file of earnings call transcripts and use the existing library APIs to:
- Build a `Transcripts` object directly from the local file
- Alternatively, use `Ticker` while pointing the SQL at the local file


In [3]:
# Setup: install deps and fix typing_extensions for OpenAI
import sys
print("Interpreter:", sys.executable)

# Install project requirements
%pip install -r "/Users/devishadhanuka/Documents/Adv NLP/defeatbeta-api-main/requirements.txt" -q

# Ensure typing_extensions is recent enough for OpenAI
%pip install -U typing_extensions==4.12.2 -q

# Sanity check versions
import typing_extensions as te
import importlib
print("typing_extensions:", getattr(te, "__version__", "unknown"))
try:
    import openai
    print("openai:", getattr(openai, "__version__", "unknown"))
except Exception as e:
    print("openai import still failing:", e)


Interpreter: /opt/anaconda3/envs/myenv/bin/python
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1 requires sympy==1.13.1, but you have sympy 1.14.0 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic 2.12.3 requires typing-extensions>=4.14.1, but you have typing-extensions 4.12.2 which is incompatible.
pydantic-core 2.41.4 requires typing-extensions>=4.14.1, but you have typing-extensions 4.12.2 which is incompatible.
torch 2.5.1 requires sympy==1.13.1, but you have sympy 1.14.0 which is incompatible.

[notice] A new release 

In [4]:
from defeatbeta_api.client.duckdb_client import get_duckdb_client
from defeatbeta_api.data.transcripts import Transcripts

# Adjust these as needed
TICKER = "AAPL"
LOCAL_PARQUET = "/Users/devishadhanuka/Documents/Adv NLP/defeatbeta-api-main/stock_earning_call_transcripts.parquet"

# Read local Parquet for the symbol
duck = get_duckdb_client()
df = duck.query(f"SELECT * FROM '{LOCAL_PARQUET}' WHERE symbol = '{TICKER}'")

# Build Transcripts object
transcripts = Transcripts(TICKER, df, log_level="INFO")
transcripts


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/devishadhanuka/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


______      __           _    ______      _        
|  _  \    / _|         | |   | ___ \    | |       
| | | |___| |_ ___  __ _| |_  | |_/ / ___| |_ __ _ 
| | | / _ \  _/ _ \/ _` | __| | ___ \/ _ \ __/ _` |
| |/ /  __/ ||  __/ (_| | |_  | |_/ /  __/ || (_| |
|___/ \___|_| \___|\__,_|\__| \____/ \___|\__\__,_|
📈:: Data Update Time ::	2025-10-24 ::
📈:: Software Version ::	0.0.24      ::


Transcripts()

In [7]:
# List available transcript quarters and pretty-print one
print(transcripts)

# Replace with a quarter that exists in your data
FISCAL_YEAR = 2023
FISCAL_QUARTER = 2

transcripts.print_pretty_table(FISCAL_YEAR, FISCAL_QUARTER)


   symbol  fiscal_year  fiscal_quarter report_date
0    AAPL         2005               4  2005-10-13
1    AAPL         2006               1  2006-01-19
2    AAPL         2006               2  2006-04-20
3    AAPL         2006               3  2006-07-19
4    AAPL         2006               4  2006-10-19
5    AAPL         2007               1  2007-01-17
6    AAPL         2007               2  2007-04-25
7    AAPL         2007               3  2007-07-25
8    AAPL         2007               4  2007-10-22
9    AAPL         2008               1  2008-01-22
10   AAPL         2008               2  2008-04-23
11   AAPL         2008               3  2008-07-21
12   AAPL         2008               4  2008-10-21
13   AAPL         2009               1  2009-01-21
14   AAPL         2009               2  2009-04-23
15   AAPL         2009               3  2009-07-21
16   AAPL         2009               4  2009-10-19
17   AAPL         2010               1  2010-01-25
18   AAPL         2010         

In [6]:
# Alternative: Use Ticker but point SQL to the local Parquet
from defeatbeta_api.data.ticker import Ticker
from defeatbeta_api.data.sql.sql_loader import load_sql_query

T = Ticker(TICKER)
sql = load_sql_query("_query_data", ticker=T.ticker, url=LOCAL_PARQUET)
df_local = T.duckdb_client.query(sql)
transcripts_alt = Transcripts(T.ticker, df_local, T.log_level)
print(transcripts_alt)
transcripts_alt.print_pretty_table(FISCAL_YEAR, FISCAL_QUARTER)


   symbol  fiscal_year  fiscal_quarter report_date
0    AAPL         2005               4  2005-10-13
1    AAPL         2006               1  2006-01-19
2    AAPL         2006               2  2006-04-20
3    AAPL         2006               3  2006-07-19
4    AAPL         2006               4  2006-10-19
5    AAPL         2007               1  2007-01-17
6    AAPL         2007               2  2007-04-25
7    AAPL         2007               3  2007-07-25
8    AAPL         2007               4  2007-10-22
9    AAPL         2008               1  2008-01-22
10   AAPL         2008               2  2008-04-23
11   AAPL         2008               3  2008-07-21
12   AAPL         2008               4  2008-10-21
13   AAPL         2009               1  2009-01-21
14   AAPL         2009               2  2009-04-23
15   AAPL         2009               3  2009-07-21
16   AAPL         2009               4  2009-10-19
17   AAPL         2010               1  2010-01-25
18   AAPL         2010         